In [10]:
import pandas as pd
import numpy as np
import math

# Read data
These files contain 1,000,209 anonymous ratings of approximately 3,900 movies made by 6,040 MovieLens users who joined MovieLens in 2000.

## Reading users file
User information is in the file "users.dat" and is in the following format:

`UserID::Gender::Age::Occupation::Zip-code`

All demographic information is provided voluntarily by the users and is not checked for accuracy. 
Only users who have provided some demographic information are included in this data set.

- Gender is denoted by a "M" for male and "F" for female
- Age is chosen from the following ranges:
    - 1:  "Under 18"
    - 18:  "18-24"
    - 25:  "25-34"
    - 35:  "35-44"
    - 45:  "45-49"
    - 50:  "50-55"
    - 56:  "56+"

- Occupation is chosen from the following choices:
    - 0:  "other" or not specified
    - 1:  "academic/educator"
    - 2:  "artist"
    - 3:  "clerical/admin"
    - 4:  "college/grad student"
    - 5:  "customer service"
    - 6:  "doctor/health care"
    - 7:  "executive/managerial"
    - 8:  "farmer"
    - 9:  "homemaker"
    - 10:  "K-12 student"
    - 11:  "lawyer"
    - 12:  "programmer"
    - 13:  "retired"
    - 14:  "sales/marketing"
    - 15:  "scientist"
    - 16:  "self-employed"
    - 17:  "technician/engineer"
    - 18:  "tradesman/craftsman"
    - 19:  "unemployed"
    - 20:  "writer"

In [2]:
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
df_users = pd.read_csv('data/users.dat', sep='::', names=u_cols, encoding='latin-1')

df_users.head()

/Users/hobey/Envs/DADO_python/lib/python3.6/site-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


,user_id,age,sex,occupation,zip_code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


## Reading items file
Movie information is in the file "movies.dat" and is in the following format:

`MovieID::Title::Genres`

- Titles are identical to titles provided by the IMDB (including year of release)
- Genres are pipe-separated and are selected from the following genres:

    - Action
    - Adventure
    - Animation
    - Children's
    - Comedy
    - Crime
    - Documentary
    - Drama
    - Fantasy
    - Film-Noir
    - Horror
    - Musical
    - Mystery
    - Romance
    - Sci-Fi
    - Thriller
    - War
    - Western

- Some MovieIDs do not correspond to a movie due to accidental duplicate
entries and/or test entries
- Movies are mostly entered by hand, so errors and inconsistencies may exist


In [3]:
i_cols = ['movie_id', 'title', 'genres']
df_movies = pd.read_csv('data/movies.dat', sep='::', names=i_cols, encoding='latin-1')

df_movies.head()

/Users/hobey/Envs/DADO_python/lib/python3.6/site-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


## Reading ratings file
All ratings are contained in the file "ratings.dat" and are in the
following format:

`UserID::MovieID::Rating::Timestamp`

- UserIDs range between 1 and 6040 
- MovieIDs range between 1 and 3952
- Ratings are made on a 5-star scale (whole-star ratings only)
- Timestamp is represented in seconds since the epoch as returned by time(2)
- Each user has at least 20 ratings


In [4]:
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
df_ratings = pd.read_csv('data/ratings.dat', sep='::', names=r_cols,encoding='latin-1')
df_ratings.head()



/Users/hobey/Envs/DADO_python/lib/python3.6/site-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


# Criteria

## Recall 召回率
描述有多少比例的用户\-物品评分记录包含在最终的推荐列表中

In [6]:
def recall(train, test, N):
    hit = 0
    total = 0
    for user in train.keys():
        tu = test[user]
        rank = get_recommendation(user, N)
        for item, pui in rank:
            if item in tu:
                hit += 1
        total += len(tu)
    
    return hit/ (total * 1.0)


## Precision 准确率
描述最终的推荐列表中有多少比例是发生过用户-物品评分记录

In [7]:
def percision(train, test, N):
    hit = 0
    total = 0
    for user in train.keys():
        tu = test[user]
        rank = get_recommendation(user, N)
        for item, pui in rank:
            if item in tu:
                hit += 1
        total += N
    
    return hit / (total * 1.0)

## Coverage 覆盖率
反映了推荐算法发掘长尾的能力

In [8]:
def coverage(train, test, N):
    recommend_items = set()
    all_items = set()
    for user in train.keys():
        for item in train[user].keys():
            all_item.add(item)
            
        rank = get_recommendation(user, N)
        
        for item, pui in rank:
            recommend_items.add(item)
    
    return len(recommend_items) / (len(all_item) * 1.0)




## Popularity 新颖度
- 推荐列表中物品的平均流行度度量推荐结果的新颖度
- 如果推荐出的物品很热门，说明新颖度很低

In [ ]:
def popularity(train, test, N):
    item_popularity = dict()
    for user, items in train.items():
        for item in items.keys():
            if item not in item_popularity:
                item_popularity[item] = 0
            item_popularity[item] += 1
    ret = 0
    n = 0
    for user in train.keys():
        rank = get_recommendation(user, N)
        for item, pui in rank:
            ret += math.log(1 + item_popularity[item])
            n += 1
    ret /= n * 1.0
    
    return ret